In [4]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\arnaz\anaconda3\lib\site-packages (0.0)


In [2]:
!pip install tensorflow

In [5]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [6]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(42)

In [7]:
# Dependencies
import numpy as np
import pandas as pd

In [8]:
import tensorflow
tensorflow.keras.__version__

'2.7.0'

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler ,OneHotEncoder
from tensorflow.keras.utils import to_categorical

In [10]:
# Need to add:
from sklearn.datasets import make_classification

X, y = make_classification(n_features=40, n_redundant=0, n_informative=40,
                           random_state=42, n_classes=2, n_clusters_per_class=1)

y = y.reshape(-1, 1)

print(X.shape)
print(y.shape)

(100, 40)
(100, 1)


# Read the CSV and Perform Basic Data Cleaning

In [11]:
df = pd.read_csv("data/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

Over a period of nine years in deep space, the NASA Kepler space telescope has been out on a planet-hunting mission to discover hidden planets outside of our solar system.

To help process this data, you will create machine learning models capable of classifying candidate exoplanets from the raw dataset.

https://exoplanetarchive.ipac.caltech.edu/docs/API_kepcandidate_columns.html


In [ ]:
kepoi_name: A KOI is a target identified by the Kepler Project that displays at least one transit-like sequence within Kepler time-series photometry that appears to be of astrophysical origin and initially consistent with a planetary transit hypothesis
kepler_name: [These names] are intended to clearly indicate a class of objects that have been confirmed or validated as planets—a step up from the planet candidate designation.
koi_disposition: The disposition in the literature towards this exoplanet candidate. One of CANDIDATE, FALSE POSITIVE, NOT DISPOSITIONED or CONFIRMED.
koi_pdisposition: The disposition Kepler data analysis has towards this exoplanet candidate. One of FALSE POSITIVE, NOT DISPOSITIONED, and CANDIDATE.
koi_score: A value between 0 and 1 that indicates the confidence in the KOI disposition. For CANDIDATEs, a higher value indicates more confidence in its disposition, while for FALSE POSITIVEs, a higher value indicates less confidence in that disposition.

In [ ]:
# Set features. This will also be used as your x values.
#selected_features = df[['names', 'of', 'selected', 'features', 'here']]

In [12]:
df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

In [13]:
#selected_features = df[['names', 'of', 'selected', 'features', 'here']]
# this is x-axis
selected_features = df[[ 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag']]

In [ ]:
selected_features.head()

In [ ]:
df.head()

In [ ]:
# drop colums koi_disposition''
#axis=1 – Specifies the axis to be deleted. Axis 1 means column and 0 means rows.
#specifies the drop operation to be in same dataframe rather creating a copy of the dataframe after drop.
#selected_features = df.drop(columns=['koi_disposition', axis=1 ,inplace=True)])
#df = df.drop(columns=["koi_disposition"],axis=1 )
#selected_features = df.drop(columns=["koi_disposition"]))


In [14]:
df.describe()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
count,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6.991000e+03,6.991000e+03,6991.000000,6991.000000,6991.000000,...,6991.000000,6991.000000,6991.000000,6991.00000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000
mean,0.157059,0.244743,0.202975,0.125018,56.191248,1.851122e-03,-1.851122e-03,164.488820,0.009340,-0.009340,...,-161.206980,4.305049,0.121091,-0.14048,1.740749,0.352710,-0.388568,292.082406,43.812143,14.271508
std,0.363882,0.429966,0.402243,0.330763,117.570962,7.184503e-03,7.184503e-03,67.020475,0.021989,0.021989,...,71.448481,0.439238,0.132048,0.08199,5.903415,0.839017,1.907797,4.762908,3.606167,1.350802
min,0.000000,0.000000,0.000000,0.000000,0.259820,1.100000e-08,-1.568000e-01,120.515914,0.000009,-0.569000,...,-1733.000000,0.047000,0.000000,-1.00700,0.109000,0.000000,-103.825000,279.856080,36.577381,6.966000
25%,0.000000,0.000000,0.000000,0.000000,2.620126,5.005000e-06,-2.401000e-04,132.683917,0.001145,-0.010000,...,-197.000000,4.209000,0.044000,-0.19500,0.829000,0.128000,-0.252000,288.704730,40.797760,13.455000
50%,0.000000,0.000000,0.000000,0.000000,8.947426,3.300000e-05,-3.300000e-05,136.739230,0.003990,-0.003990,...,-159.000000,4.436000,0.070000,-0.12700,0.999000,0.248000,-0.111000,292.314760,43.679661,14.534000
75%,0.000000,0.000000,0.000000,0.000000,34.282605,2.401000e-04,-5.005000e-06,169.937005,0.010000,-0.001145,...,-112.000000,4.543000,0.149000,-0.08800,1.357000,0.357000,-0.069000,295.888550,46.693659,15.322000
max,1.000000,1.000000,1.000000,1.000000,1071.232624,1.568000e-01,-1.100000e-08,1472.522306,0.569000,-0.000009,...,0.000000,5.364000,1.472000,0.00000,180.013000,25.956000,0.000000,301.720760,52.336010,19.065000


In [15]:
X=df.drop("koi_disposition",axis=1)
y= df["koi_disposition"]
print(X.shape,y.shape)

(6991, 40) (6991,)


# Create a Train Test Split

Use `koi_disposition` for the y values

In [16]:
# x = selected_features # Known 
# y = unknown
y.value_counts()

FALSE POSITIVE    3504
CONFIRMED         1800
CANDIDATE         1687
Name: koi_disposition, dtype: int64

In [19]:
X=selected_features
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)


In [20]:
y_train.head()

6122         CANDIDATE
6370    FALSE POSITIVE
2879    FALSE POSITIVE
107          CONFIRMED
29           CANDIDATE
Name: koi_disposition, dtype: object

In [ ]:
y_test.count()

In [ ]:
y_train.count()


In [ ]:
X_train.count()

In [ ]:
X_train.count()

In [ ]:
X_test.count()

predictions = model_LogisticRegression.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [ ]:
# Scale your data

In [21]:
#Cut and paste without understanding
from sklearn.preprocessing import  MinMaxScaler

# Create a StandardScater model and fit it to the training data

### BEGIN SOLUTION
X_scaler =  MinMaxScaler().fit(X_train)

### END SOLUTION

In [22]:
#Cut and paste without understanding
### BEGIN SOLUTION
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


### END SOLUTION

In [23]:
#convert from categorical to numeric.
# Y is a target value but not in numeric
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)

In [24]:
#convert from categorical to numeric.
# Y is a target value but not in numeric
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_test)
encoded_y_test = label_encoder.transform(y_test)

In [25]:
#Check
for label, original_class in zip(encoded_y_test, y):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CO

Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 0


In [26]:
#Check
for label, original_class in zip(encoded_y_train, y):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: CONFIRMED
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 0
------------
Original Class: CO

------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 0
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 0
------------
Original Class: CANDIDATE
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: FALSE POSITIVE

Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 1
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 1
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Ori

Original Class: FALSE POSITIVE
Encoded Label: 1
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 1
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 0
------------
Original Class: CONFIRMED
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 0
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 1
------------
Original Class: CANDIDATE
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 0
------------
Original

Encoded Label: 1
------------
Original Class: CANDIDATE
Encoded Label: 1
------------
Original Class: CANDIDATE
Encoded Label: 1
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: CANDIDATE
Encoded Label: 1
------------
Original Class: FALSE POSITIVE
Encoded Label: 1
------------
Original Class: CANDIDATE
Encoded Label: 2
------------
Original Class: CONFIRMED
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 0
------------
Original Class: CANDIDATE
Encoded Label: 0
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIVE
Encoded Label: 2
------------
Original Class: FALSE POSITIV

In [27]:
from tensorflow.keras.utils import to_categorical
# One-hot encoding
# 
y_train_categorical = to_categorical(encoded_y_train)
y_train_categorical

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [28]:
y_test_categorical = to_categorical(encoded_y_test)
y_test_categorical

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [29]:
from tensorflow.keras.models import Sequential

model = Sequential() 

In [30]:
##Question
from tensorflow.keras.layers import Dense
number_inputs = 40
number_hidden_nodes = 4
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

In [31]:
number_classes = 3
model.add(Dense(units=number_classes, activation='softmax'))

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 164       
                                                                 
 dense_1 (Dense)             (None, 3)                 15        
                                                                 
Total params: 179
Trainable params: 179
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [34]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
164/164 - 0s - loss: 0.9998 - accuracy: 0.4846 - 487ms/epoch - 3ms/step
Epoch 2/60
164/164 - 0s - loss: 0.8344 - accuracy: 0.5741 - 123ms/epoch - 753us/step
Epoch 3/60
164/164 - 0s - loss: 0.6691 - accuracy: 0.7442 - 130ms/epoch - 790us/step
Epoch 4/60
164/164 - 0s - loss: 0.5591 - accuracy: 0.7496 - 124ms/epoch - 759us/step
Epoch 5/60
164/164 - 0s - loss: 0.4903 - accuracy: 0.7536 - 133ms/epoch - 811us/step
Epoch 6/60
164/164 - 0s - loss: 0.4485 - accuracy: 0.7644 - 132ms/epoch - 808us/step
Epoch 7/60
164/164 - 0s - loss: 0.4232 - accuracy: 0.7719 - 131ms/epoch - 800us/step
Epoch 8/60
164/164 - 0s - loss: 0.4082 - accuracy: 0.7755 - 128ms/epoch - 782us/step
Epoch 9/60
164/164 - 0s - loss: 0.3985 - accuracy: 0.7923 - 135ms/epoch - 821us/step
Epoch 10/60
164/164 - 0s - loss: 0.3925 - accuracy: 0.8011 - 128ms/epoch - 778us/step
Epoch 11/60
164/164 - 0s - loss: 0.3881 - accuracy: 0.7910 - 125ms/epoch - 759us/step
Epoch 12/60
164/164 - 0s - loss: 0.3846 - accuracy: 0.7994 - 124m

## Question should y be encoded_y
#model_LogisticRegression.fit(X_train_scaled , encoded_y)
model_LogisticRegression.fit(X_train_scaled , y_train)

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)